In [33]:
import tensorflow as tf
import numpy as np 
import pandas as pd

In [92]:
score=pd.read_csv('score.csv',header=None)
job_feature=pd.read_csv('job_feature.csv',index_col=False)
query_feature=pd.read_csv('query_feature.csv')

In [93]:
score.columns=['jobno','querystring','y']

In [94]:
job_feature=job_feature.iloc[:,1:]

In [101]:
query_feature=query_feature.iloc[:,1:]

In [103]:

def get_variable(lis,query):
    #1 把list中每個元素都找到job_feature並與query join
    #2 計算出每個query對應到job的rel分數，若沒有值補0
    d = {'jobno': lis, 'querystring': [query for i in lis]}
    dat = pd.merge(pd.DataFrame(d), score, how='left', on=['jobno','querystring']).fillna(0)
    #3 merge-job feature
    dat = pd.merge(pd.DataFrame(dat), job_feature, how='left', on=['jobno'])
    #4 merge query feature
    dat = pd.merge(pd.DataFrame(dat), query_feature, how='left', on=['querystring'])
    
    return dat

In [105]:
dd=get_variable([9999831,10000054,10000112,],'keyword=sales%20assistance&jobsource=n104bank1&ro=0&order=1')

In [113]:
dd.iloc[:,3:].shape

(3, 31)

In [122]:
class ListNet():
    def __init__(self,learning_rate=0.001,list_size=3,query_feature_size=31,job_feature_size=0):
        tf.reset_default_graph()
        self.learning_rate = learning_rate
        self.list_size = list_size 
        self.query_feature_size=query_feature_size
        self.job_feature_size=job_feature_size
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [self.list_size, self.query_feature_size+self.job_feature_size])
        self.y = tf.placeholder(tf.float32, [self.list_size, ])
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.global_variables_initializer()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        # encoder -> 把x encode成 z ->把z encode成mu,sigma 
        network_weights=self._initialize_weights()
        
        layer1=tf.add(tf.matmul(self.x,network_weights['weights']['Deep1']),network_weights['b']['Deep1'])
        layer1=tf.sigmoid(layer1)
        
        layer2=tf.add(tf.matmul(layer1,network_weights['weights']['Deep2']),network_weights['b']['Deep2'])
        layer2=tf.sigmoid(layer2)
        
        self.t=tf.add(tf.matmul(layer2,network_weights['weights']['out']),network_weights['b']['out'])
        
    def _initialize_weights(self):
        all_weights={
                "weights":{
                    "Deep1":tf.get_variable('C_W_1', shape=(self.query_feature_size+self.job_feature_size,10), initializer=tf.contrib.layers.xavier_initializer()),
                    "Deep2":tf.get_variable('C_W_2', shape=(10,5), initializer=tf.contrib.layers.xavier_initializer()) ,
                    "out":tf.get_variable('out_w', shape=(5,1), initializer=tf.contrib.layers.xavier_initializer()) 
                },
                "b":{
                    "Deep1":tf.get_variable('C_b_1', shape=(10), initializer=tf.contrib.layers.xavier_initializer()),
                    "Deep2":tf.get_variable('C_b_2', shape=(5), initializer=tf.contrib.layers.xavier_initializer()) ,
                    "out":tf.get_variable('out_b', shape=(1), initializer=tf.contrib.layers.xavier_initializer()) 
                }
            }
        return all_weights
    
    def _create_loss_optimizer(self):
        real_score=tf.divide(tf.exp(self.y),tf.reduce_sum(tf.exp(self.y)))
        model_score=tf.divide(tf.exp(self.t),tf.reduce_sum(tf.exp(self.t)))
        self.cost = -tf.reduce_sum(tf.multiply(real_score,tf.log(model_score)))
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
    
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.

        Return cost of mini-batch.
        """
        #opt, cost = self.sess.run((self.optimizer, self.cost), 
    #                                   feed_dict={self.x: X})
        opt,cost=self.sess.run((self.optimizer,self.cost),{self.x:X})
        return cost
    
    
        

In [123]:
a=ListNet()

In [131]:
a.sess.run(a.optimizer,feed_dict={a.x:np.array(dd.iloc[:,3:]),a.y:np.array(dd['y'])})